In [22]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import preprocessing
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from identification import vif_detection

In [23]:
### Load in Data ###
data = pd.read_csv("data_cleaning/final_data.csv")


vars = ["REGION_YEAR","AGELAST","SEX","RACETHX","MARRY_YEARX","EDUCYR",
"BORNUSA","FOODST_YEAR","TTLP_YEARX","FAMINC_YEAR","POVCAT_YEAR","POVLEV_YEAR","WAGEP_YEARX",
"DIVDP_YEARX","SALEP_YEARX","PENSP_YEARX","PUBP_YEARX","ADHDADDX","ACTDTY",
'UNINSURED_ONLY', 'PRIVATE_ONLY', 'MEDICAID_ONLY', 'MEDICARE_ANY', 'MEDICARE_ADV', 'MEDICARE_MEDICAID', 'MEDICARE_PRIVATE',	
"RTHLTH","MNHLTH","EMPST","non_opioid_prescriptions","NUM_CONDITIONS","INJURY"]

data = data.dropna()

### Run identification ###
y = pd.DataFrame(data, columns=['opioid_prescribed_at_all'])
exog = pd.DataFrame(data, columns=vars)
exog_vars = vif_detection(data,exog, y)

### Data Normalization and Splitting ###
X=pd.DataFrame(exog_vars).to_numpy()
y=pd.DataFrame(y).to_numpy().reshape(len(y),)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_scaled = scaler.transform(X)
X_train, X_test_valid, y_train, y_test_valid = train_test_split(X_scaled, y, random_state=42, test_size = 0.2)
X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid, random_state=42, test_size = 0.5)

In [24]:
### Model 1-Logistic Regression ###
log_reg= LogisticRegression(max_iter = 1000).fit(X_train, y_train)  # apply scaling on training data
score = log_reg.score(X_valid, y_valid)
print(f"logistic model accuracy on valid: {score*100:.1f}%")

logistic model accuracy on valid: 82.2%


/Users/eujeneyum/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
### Model 2-Decision Tree ###
decision_tree = tree.DecisionTreeClassifier()
decision_tree = decision_tree.fit(X_train, y_train)
score = decision_tree.score(X_valid, y_valid)
print(f"tree accuracy on valid: {score*100:.1f}%")

tree accuracy on valid: 73.6%


In [26]:
### Model 3-Random Forest ###
random_forest_scores = []
for estimator in [10,50,100,250,500]:
    forest_model = RandomForestClassifier(random_state = 0, n_jobs = 1, n_estimators = 100, class_weight = 'balanced')
    forest_model = forest_model.fit(X_train, y_train)
    score = forest_model.score(X_valid, y_valid)
    random_forest_scores.append(score)
    print(f"random forest with {estimator} estimators' accuracy on valid: {score*100:.1f}%")

random forest with 10 estimators' accuracy on valid: 83.3%
random forest with 50 estimators' accuracy on valid: 83.3%
random forest with 100 estimators' accuracy on valid: 83.3%
random forest with 250 estimators' accuracy on valid: 83.3%
random forest with 500 estimators' accuracy on valid: 83.3%


In [27]:
### Model 4-Neural Net ###
nn_scores = []
for layers in [(100,50,25), (10, 5, 2), (25, 25, 25), (10, 9, 8), (10, 5), (100, 25)]:
    for active_func in ['relu', 'tanh', 'logistic']:
        for alpha_val in [0.0001, 0.001, .01]:
            for max_iterations in [100, 200, 500]:
                for type_of_solver in ['sgd', 'adam']:
                    if type_of_solver == 'sgd':
                        clf = MLPClassifier(hidden_layer_sizes = layers,
                                            activation = active_fun,
                                            alpha = alpha_val,
                                            solver = type_of_solver,
                                            learning_rate = 'adaptive',
                                            max_iter = max_iterations,
                                            shuffle = True,
                                            random_state=1).fit(X_train, y_train)
                    if type_of_solver == 'adam':
                        clf = MLPClassifier(hidden_layer_sizes = layers,
                                            activation = active_fun,
                                            alpha = alpha_val,
                                            solver = type_of_solver,
                                            max_iter = max_iterations,
                                            shuffle = True,
                                            random_state=1).fit(X_train, y_train)
                    score = clf.score(X_valid, y_valid)
                    nn_scores.append(score)
                    print(f"neural net with {layers} layers, {active_func} activation,\
                        {alpha_val} alpha, {max_iterations} max iternations, and \
                        {type_of_solver} type of solver accuracy on valid: {score*100:.1f}%")

NameError: name 'active_fun' is not defined

In [ ]:
### Model 5-Ada Boost ###
ada_boost_scores = []
for learning_rate in [0.01, .05, 0.1, 0.2]:
    for n in [100, 200, 500, 1000]:
        ada_boost = AdaBoostClassifier(n_estimators=n, learning_rate=learning_rate, algorithm='SAMME.R', random_state=1).fit(X_train, y_train)
        score = ada_boost.score(X_valid, y_valid)
        ada_boost_scores.append(score)
        print(f"Ada accuracy with learning rate of {learning_rate} and number estimators {n} on valid: {score*100:.1f}%")